In [ ]:
# Instalamos los paquetes necesarios
%pip install pysqlite3
%pip install chromadb

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import os
import chromadb
import pandas as pd
pd.set_option('max_colwidth', 1000)
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [5]:
# Seteamos variables
chroma_db_path = "../chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small"
collection_name = "Montes_del_Plata"
directory = '../documents'
OPENAI_API_KEY = ''

In [13]:
# Creamos una base de datos vectorial local con Chroma
client_chroma = chromadb.PersistentClient(path=chroma_db_path)
embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=EMBEDDING_MODEL)
collection = client_chroma.create_collection(name=collection_name, 
                                             metadata={"hnsw:space": "cosine"}, 
                                             embedding_function=embedding_function)

In [14]:
# Leemos los documentos a guardar en la base de datos vectorial

docs_name = ['document_0', 'document_1', 'document_2', 'document_3', 'document_4', 'document_5', 'document_6', 'document_7', 'document_8',
 'document_9', 'document_10', 'document_11', 'document_12', 'document_13', 'document_14', 'document_15_1', 'document_15_2', 'document_15_3', 
 'document_15_4', 'document_15_5', 'document_15_6', 'document_15_7', 'document_15_8', 'document_16_1', 'document_16_2', 'document_17_1', 
 'document_17_2', 'document_17_3', 'document_17_4', 'document_18_1', 'document_18_2', 'document_18_3', 'document_19'] 
documents = {}

for doc in docs_name:
    file_path = os.path.join(directory, doc+".txt")
    with open(file_path, 'r') as file:
        documents[doc] = file.read()

In [15]:
# Creamos metadata para guardar con los documentos
# la estructura de los datos es una lista de diccionarios
metadata = []

departamento = ['Forestal']*12 + ['Planta']*(len(documents.keys())-12)
nombre_doc = ['P_LOGT_01: TRANSPORTE DE MADERA']*12 + ['Instrucciones de operación de procesos: Blanqueo']*(len(documents.keys())-12)
capitulo = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '0', '1', '2', '3', '3', '3', '3', '3', '3', '3', '3', '4', '4', '5', '5', '5', '5', '6', '6', '6', '7']

# las 3 listas anteriores deberian tener el mismo largo
for i in range(len(departamento)):
    metadata.append(
        {'departamento' : departamento[i],
         'nombre_documento' : nombre_doc[i],
         'capitulo' : capitulo[i]}
    )

In [16]:
# Almacenamos los datos en la colección
# Dejamos que los embeddings de cada documento los genere Chroma 
# haciendo una llamada al modelo de embedding de OpenAI que elegimos al comienzo

collection.add(
    ids=[str(num) for num in range(len(documents.keys()))],
    documents=list(documents.values()),
    metadatas=metadata
)

In [17]:
client_chroma.list_collections()

[Collection(id=e97c9ab4-4c0d-467f-80c8-02e1d948e120, name=Montes_del_Plata)]

In [18]:
collection = client_chroma.get_collection(name="Montes_del_Plata")
collection.count()

33